# **2nd Day - Lab-2**
---
**Lab Objective:**
> In this lab, we will figure out import Cleverhans to run adversarial attacks against CNN network. In addition, some adversarial attacks will be implemented. Finally, we will evaluate the transferability of  adversarial examples from model A to model B.
---
# **PART-2:**

In this part we will load MNIST dataset and train  TWO CNN models with it. Then, we will run FGSM attack again the 1st model. Then, we feed the generated example into the second model and show the result.

---

# Part1-Step1 : **Import required libaries and MNIST dataset**

In [ ]:
!pip install -q tensorflow==2.0.0b1
# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
import cleverhans
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# Part1-Step2 : 

In this step, two different CNN networks are created and trained with MNIST dataset. 

In [ ]:
num_classes = 10
model_1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation(tf.nn.softmax) # We seperate the activation layer to be able to access the logits of the previous layer later
])
model_1.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',  metrics=['accuracy'])
model_1.fit(train_images, train_labels, epochs=10, validation_split=0.2)
model_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation(tf.nn.softmax) # We seperate the activation layer to be able to access the logits of the previous layer later
])
model_2.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',  metrics=['accuracy'])
model_2.fit(train_images, train_labels, epochs=10, validation_split=0.2)

# Part1-Step3 : 
---
In this step, we select a random image and using "fast_gradient_method", we run attack against model_1. Then, we feed the generated attack into the model_2 (transfer the attack) and show the result as well.

In [ ]:
# Import the attack
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
random_index = 7204 #a random index to select sample and make it as adversarial example
original_image = test_images[random_index]
original_image = tf.convert_to_tensor(original_image.reshape((1,28,28))) #The .reshape just gives it the proper form to input into the model, a batch of 1 a.k.a a tensor
original_label=(test_labels[random_index])


logits_model = tf.keras.Model(model_1.input,model_1.layers[-1].output) #The attack requires the model to ouput the logits . Logit is last layer of CNN model 
epsilon = 0.1 #a parameter to control the size of modification (purturbation) on original image to convert it to adversarial example.
adv_example= fast_gradient_method(logits_model, original_image, epsilon, np.inf, targeted=False)#Here the adversarial example is generated
adv_example_label_model_1 = model_1.predict(adv_example) 
adv_example_label_model_2 = model_2.predict(adv_example) 
print('>> Adversarial Example is Created for images ', random_index)

# Here, we visulize original and adversarial example
plt.figure()
plt.imshow(np.reshape(original_image, (28,28)))
plt.title("Original Image Label: {}".format(original_label))
plt.show()
plt.imshow(np.reshape(adv_example, (28,28)))
plt.title("Adversarial Image, Model_1 Prediction: {}".format(np.argmax(adv_example_label_model_1)))
plt.show()
plt.imshow(np.reshape(adv_example, (28,28)))
plt.title("Adversarial Image, Model_2 Prediction: {}".format(np.argmax(adv_example_label_model_2)))
plt.show()
